# Overview
In this notebook, we apply a parameter estimation method on Kholodenko's model of EGFR signalling pathway. This involves performing parameter estimation, then assessing the quality of the optimized model.

The model that is taken from [Kholodenko et. al (1999)](https://www.sciencedirect.com/science/article/pii/S0021925819518804), and has already been implemented in the rule-based modeling using BioNetGen and also in the
systems biology model specification format (SBML).

The model in Step 4 recovers the Kholodenko's output, but the results of the Kholodenko's model do not fit the experimental data very well, as we can see in the following figure. As a result, in order to discover a set of optimized parameters, I employed a parameter estimation method for the model in Step 4. 

The set of optimized parameters seeks to provide outputs which best follow the experimental data in Kholodenko's paper. 
</ul>
In this figure, you can see the Kholodenko's model output vs experimental data:


</ul>
<br>

![KholodenkoModel.png](KholodenkoModel.png)
<br><br>

## Problem Specification, Import, and Setup

In [ ]:
import petab
import pypesto
import pypesto.optimize as optimize
import pypesto.petab
import pypesto.sample as sample
import pypesto.visualize as visualize
import amici
from pypesto.store import read_from_hdf5, save_to_hdf5
import matplotlib.pyplot as plt
import numpy as np

The parameter estimation problem has also already been formulated in [PEtab](https://github.com/PEtab-dev/PEtab) [3].
The PEtab format is compatible with a variety of tools that are primarily developed within the systems biology community. Here, the [pyPESTO](https://github.com/ICB-DCM/pyPESTO) tool is for parameter estimation. 

The default simulation tool used by pyPESTO is [AMICI](https://github.com/AMICI-dev/AMICI).



In [ ]:
petab_problem = petab.Problem.from_yaml(
    
   "../EGFR/EGFR.yaml"    #state the exact folder contains the yaml file
)
importer = pypesto.petab.PetabImporter(petab_problem)
# import to pypesto
problem = importer.create_problem()
model = importer.create_model(verbose=False)

## Parameter Estimation

A multi-start optimization is used here, to efficiently explore the parameter space for optima. The author' experience with the difficulty of optimizing this problem led her to use the [Fides](https://github.com/fides-dev/fides) optimizer.
The choice of number of starts is problem-specific.

In [ ]:
# create optimizer object which contains all information for doing the optimization
options = {'maxiter':2000}
optimizer = optimize.FidesOptimizer(options=options)
engine = pypesto.engine.MultiProcessEngine()

# do the optimization
result = optimize.minimize(
    problem=problem, optimizer=optimizer, n_starts=100, engine=engine
)

## Visualization and Assessment of Optimization

The first plot here is the waterfall plot, which shows the likelihood function value of the estimated parameter values at the end of each optimization run (start). The runs are ordered by likelihood function value. Generally, a plateau of a few starts suggests a successful optimization with the good global optima found. Other colors are used here to indicate plateaus showing different local optimums.

If the waterfall plot does not show a plateau at the minimum, the bounds can be adjusted (preferably to more realistic bounds), or the optimization can be run with a higher number of starts, or a different optimization method.

In [ ]:
visualize.waterfall(result)

<br>

![waterfallplot](waterfallplot.png)
<br><br>

The second plot is the parameters plot, which shows the estimated parameter values for each parameter at the end of each start. The vector of parameter values from a single start is indicated by connected dots. 


In [ ]:
visualize.parameters(result)

<br>

![Parametersplot](Parametersplot.png)
<br><br>

We can also conveniently visualize the model fit. This plots the petab visualization using optimized parameters.

In [ ]:
from pypesto.visualize.model_fit import visualize_optimized_model_fit
pp1 = visualize_optimized_model_fit(petab_problem=petab_problem, result=result)

</ul>
<br>

![Optimizationplot](Optimizationplot.png)
<br><br>

## Negative or positive cooperativity of ShcP in the model:

Based on the estimated parameters, it seems that the affinity of Shc for binding to EGFR intensively decreases after phosphorylation.

### v13: EGFR + Shc
```
K13 = 1.7e-4
```
### v15: EGFRShcP Breaking apart
```
K15 = 3.3e-11
```
### Cooperativity of ShcP in the original Kholodenko model
For original Kholodenko model we have:
```
K13 = 0.15
```
```
K15 = 0.003
```
Based on these values, in the original Kholodenko’s model, the affinity of Shc for binding to EGFR decreases after phosphorylation.



# Assessment of Maximum Likelihood Estimate

Once optimization appears successful, the maximum likelihood estimate (MLE) can be assessed for parameter and prediction uncertainty.

Parameter uncertainty can be assessed with MCMC sampling.

## MCMC Sampling

MCMC sampling is a method of analysing the uncertainty of a parameter estimate. Here, the adaptive Metropolis-Hastings algorithm is used.

In [ ]:
mle = result.optimize_result.list[0]['x'] # Maximum likelihood from optimization

sampler = sample.MetropolisSampler({'std': 0.00005})

result = sample.sample(
    problem,
    n_samples = 2.5e6,
    sampler = sampler,
    x0 = mle
)
elapsed_time = result.sample_result.time
print(f"Elapsed time: {round(elapsed_time,2)}")

### Sampling Evaluation

First, we determine the acceptance rate of the MCMC sampling.

In [ ]:
def calc_acceptance(samp_result):
    params = samp_result['trace_x'][0]
    accept = [0 if (params[i, :] == params[i-1, :]).all() else 1 
              for i in range(1, params.shape[0])]
    num_accept = np.sum(accept)
    rate = num_accept / (params.shape[0] - 1)
    return rate

accept_rate = calc_acceptance(result.sample_result)
print("The acceptance rate of MCMC is:  %.4f%%"%(accept_rate*100))

The acceptance rate of MCMC is:  98.9083%

The acceptance rate is high and it's beacause the std value for sampling is very small (0.00005).

Every Markov chain needs a certain amount of iterations to reach the stationary distribution. Sometimes the chain quickly get to the regions with relative high density, for some situations, especially for multiparameter problems, it usually takes hundreds or thousands of iterations to get there. Iterations obtained before a Markov chain reaches the stationary distribution are called burn-in and can be determined using Geweke test.

In [ ]:
sample.geweke_test(result=result)

Geweke burn-in index: 1375000

### Visualize the result
#### Log posterior 
x and y-axis show the iterations and log posterior of function values, respectively.

In [ ]:
visualize.sampling_fval_traces(result, size = (13,4)) 

![fvaltraceplot](fvaltraceplot.png)
<br><br>

The log-posterior chain should be smoothly varying around the maximum. We can say it has been obtained in this sampling.

#### Parameter trajectories 

In [ ]:
visualize.sampling_parameter_traces(
    result, use_problem_bounds=True, size=(30,24), stepsize = 100
)

![ParameterTraces](ParameterTraces.png)
<br><br>

For two of the parameters we show the MLE and parameter traces simultaneously, which shows the samples are around the MLE, but again the variations of parameter traces are very low.

![ParameterTraces_1_hline](ParameterTraces_1_hline.png)
<br><br>

![ParameterTraces_2_hline](ParameterTraces_2_hline.png)
<br><br>

We can plot e.g. kernel density estimates or histograms.

In [ ]:
pypesto.visualize.sampling_1d_marginals(
        result, size = (30,24)
    )

![MCMCchainsplot](MCMCchainsplot.png)
<br><br>

In these histograms, the variations are very small.

# Parallel Tempering MCMC

We also use the parallel tempering MCMC sampling for this example.

In [ ]:
sampler = sample.ParallelTemperingSampler(
    internal_sampler=sample.MetropolisSampler(),  n_chains=5
)

result = sample.sample(
    problem = problem, 
    n_samples = 5e5,
    sampler = sampler,
    x0 = mle
)

elapsed_time = result.sample_result.time
print(f"Elapsed time: {round(elapsed_time,2)}")

### Evaluating the Results

In [ ]:
def calc_acceptance(samp_result):
    params = samp_result['trace_x'][0]
    accept = [0 if (params[i, :] == params[i-1, :]).all() else 1 
              for i in range(1, params.shape[0])]
    num_accept = np.sum(accept)
    rate = num_accept / (params.shape[0] - 1)
    return rate

accept_rate = calc_acceptance(result.sample_result)
print("The acceptance rate of MCMC is:  %.4f%%"%(accept_rate*100))

sample.geweke_test(result= result)

The acceptance rate of MCMC is 23.3938%

Geweke burn-in index: 325000

Since we’re using the Metropolis algorithm, we want to monitor the acceptance rate and make sure it is within optimal range. If we accept almost every time, that tells us that each time the chain only jumps a very small step (so that the acceptance ratio is close to 1 every time), which will make the algorithm slow in converging to the stationary distribution.

On the other hand, if the acceptance rate is very low, then that says that the chain got stuck to just a few locations and it takes hundreds of iterations for it to make one jump. For the Metropolis algorithm, an optimal acceptance rate would be something between 10% to 60%. 

For our model, the acceptance rate is within the optimal range.

In [ ]:
for i_chain in range(len(result.sample_result.betas)):
    visualize.sampling_fval_traces(result,  i_chain=i_chain, size = (13,4), stepsize= 100) 

### Chain 1

![fvaltrace_chain0](fvaltrace_chain0.png)


### Chain 2

![fvaltrace_chain1](fvaltrace_chain1.png)


### Chain 3

![fvaltrace_chain2](fvaltrace_chain2.png)

### Chain 4

![fvaltrace_chain3](fvaltrace_chain3.png)


### Chain 5

![fvaltrace_chain4](fvaltrace_chain4.png)
<br><br>

### Parameter Traces

In order to show the trace plots better and also make it possible to compare the traces in different chains. We have shown several parameter traces here.

In [ ]:
n_par = 30
for i_chain in range(len(result.sample_result.betas)):
    visualize.sampling_parameter_traces(
        result, i_chain=i_chain, suptitle=f"Chain: {i_chain}", par_indices = [n_par], stepsize=100
    )
    plt.axhline(y = mle[n_par], color='r')
    

![Pram_Trace_300](Pram_Trace_300.png)
![Pram_Trace_301](Pram_Trace_301.png)
![Pram_Trace_302](Pram_Trace_302.png)
![Pram_Trace_303](Pram_Trace_303.png)
![Pram_Trace_304](Pram_Trace_304.png)

In [ ]:
n_par = 24
for i_chain in range(len(result.sample_result.betas)):
    visualize.sampling_parameter_traces(
        result, i_chain=i_chain, suptitle=f"Chain: {i_chain}", par_indices = [n_par], stepsize=100
    )
    plt.axhline(y = mle[n_par], color='r')
    

![Pram_Trace_240](Pram_Trace_240.png)
![Pram_Trace_241](Pram_Trace_241.png)
![Pram_Trace_242](Pram_Trace_242.png)
![Pram_Trace_243](Pram_Trace_243.png)
![Pram_Trace_244](Pram_Trace_244.png)

In [ ]:
n_par = 29
for i_chain in range(len(result.sample_result.betas)):
    visualize.sampling_parameter_traces(
        result, i_chain=i_chain, suptitle=f"Chain: {i_chain}", par_indices = [n_par], stepsize=100
    )
    plt.axhline(y = mle[n_par], color='r')

![Pram_Trace_290](Pram_Trace_290.png)
![Pram_Trace_291](Pram_Trace_291.png)
![Pram_Trace_292](Pram_Trace_292.png)
![Pram_Trace_293](Pram_Trace_293.png)
![Pram_Trace_294](Pram_Trace_294.png)
  

The sample traces are varied around the MLE.

### Histograms of samples

In [ ]:
#Histogram of samples 
for i_chain in range(len(result.sample_result.betas)):
    visualize.sampling_1d_marginals(
        result, i_chain=i_chain, suptitle=f"Chain: {i_chain}"
    )

### Chain 1 
![MCMCchains_PT:0](MCMCchains_PT:0.png)

### Chain 2 
![MCMCchains_PT:1](MCMCchains_PT:1.png)

### Chain 3 
![MCMCchains_PT:2](MCMCchains_PT:2.png)

### Chain 4 
![MCMCchains_PT:3](MCMCchains_PT:3.png)

### Chain 5
![MCMCchains_PT:4](MCMCchains_PT:4.png)

For having a better comparison, we concentrate on two parameters:

In [ ]:
n_par = 30
for i_chain in range(len(result.sample_result.betas)):
    visualize.sampling_1d_marginals(
        result, i_chain=i_chain, suptitle=f"Chain: {i_chain}", par_indices= [n_par]
    )
    plt.axvline(x = mle[n_par], color='r')

![MCMC_V160](MCMC_V160.png)
![MCMC_V161](MCMC_V161.png)
![MCMC_V16f2](MCMC_V162.png)
![MCMC_V16f3](MCMC_V163.png)
![MCMC_V16f4](MCMC_V164.png)

It seems that 4 last chains are almost the same. The first chain is a bit different between these chains.

In [ ]:
n_par = 24
for i_chain in range(len(result.sample_result.betas)):
    visualize.sampling_1d_marginals(
        result, i_chain=i_chain, suptitle=f"Chain: {i_chain}", par_indices= [n_par]
    )
    plt.axvline(x = mle[n_par], color='r')

![MCMC_K13f0](MCMC_K13f0.png)
![MCMC_K13f1](MCMC_K13f1.png)
![MCMC_K13f2](MCMC_K13f2.png)
![MCMC_K13f3](MCMC_K13f3.png)
![MCMC_K13f4](MCMC_K13f4.png)

In [ ]:
n_par = 4
for i_chain in range(len(result.sample_result.betas)):
    visualize.sampling_1d_marginals(
        result, i_chain=i_chain, suptitle=f"Chain: {i_chain}", par_indices= [n_par]
    )
    plt.axvline(x = mle[n_par], color='r')

![MCMC_K3f0](MCMC_K3f0.png)
![MCMC_K3f1](MCMC_K3f1.png)
![MCMC_K3f2](MCMC_K3f2.png)
![MCMC_K3f3](MCMC_K3f3.png)
![MCMC_K3f4](MCMC_K3f4.png)

## Gelman-Rubin Convergence Diagnostic

Gelman and Rubin diagnostics is introduce in Gelman and Rubin (1992) to compare several chains. This approach is based on the analysis of variance. Approximate convergence is diagnosed when the variance between the different chain, B is no larger than the variance within each individual chain, W.

Then, potential scale reduction factor (PSRF) is calculated for each of the parameters.

```
PSRF = 
[1.14082463 1.23114944 1.12906588 1.01141539 1.04598493 1.99077817
 1.6706821  1.8148313  1.41068425 1.12990836 1.04368458 1.16037462
 1.61973931 1.32567647 1.01141244 1.33286995 1.36766626 1.12544081
 1.07194149 1.00064802 1.21010824 1.84223955 1.09701744 1.4204779
 1.03900114 1.38375614 1.3860118  1.24321072 1.20868943 1.14649828
 1.00108959 1.19596478 1.24719141 1.25615038 1.00647217 1.11599609
 1.0018266  1.26464576 1.00826417 1.07113752 1.11365357 1.10511415
 1.05842486 1.42407651 1.0937442  1.5660906  1.18939625 1.02829364
 1.59172539 1.54878106]
```